In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
from sqlalchemy import inspect
import sys
import sqlalchemy.ext.declarative as sqld
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
sys.path.append('..')

In [3]:
from modules.transforms import *

In [4]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [5]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [6]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [7]:
pop=pd.read_sql_table('producto1_Covid_19', schema='producto1',con=cnx) 
pop=pop.groupby('Region')['Poblacion'].sum()
popTOT=pop.sum()

In [8]:
popTOT

19458310.0

In [9]:
#----------------------------DEIS--------------------------------------------------------------------
s="DEIS"
n="DEIS_2020"
DEIS=pd.read_sql_table(n, con=cnx,schema=s)
DEIS=DEIS[DEIS['COVID'].notnull()]
#fallecidos nuevos regionales (DEIS)
fr=DEIS.groupby(['FECHA_DEF','Metropolitana'])['FECHA_STR'].count()
fr=fr.reset_index()
fr.columns=['Fecha','Region Metropolitana','Fallecidos Nuevos']
fr=fr.pivot(index='Fecha',columns=['Region Metropolitana'])
fr=fr.T.reset_index().T
fr.columns=fr.iloc[0].astype(str)+' '+fr.iloc[1].astype(str)+' (DEIS)'
fr=fr.iloc[3:].reset_index()
#fallecidos regional acumulado (deis)
fr=fr.fillna(0)
fra=fr[fr.columns.to_list()[1:]].cumsum()
fra.columns=[x.replace('Nuevos','Acumulados') for x in fra.columns]
#concatenar
fr=pd.concat([fr,fra],axis=1)

deis=DEIS.groupby('FECHA_DEF')['index'].count().reset_index()
deis.columns=['Fecha','Fallecidos DEIS Diarios']
deis=deis.merge(DEIS[DEIS['COVID']=='Covid PCR+'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (PCR+)'}),on='Fecha')
deis=deis.merge(DEIS[DEIS['COVID']=='Covid Sospechoso'].groupby('FECHA_DEF')['index'].count().reset_index().rename(columns={
    'FECHA_DEF':'Fecha','index':'Fallecidos DEIS Diarios (Covid Sospechoso)'}),on='Fecha')
deis['Fallecidos DEIS (PCR+) Acumulado']=deis['Fallecidos DEIS Diarios (PCR+)'].cumsum()
deis['Fallecidos DEIS (Covid Sospechoso) Acumulado']=deis['Fallecidos DEIS Diarios (Covid Sospechoso)'].cumsum()
deis['Fallecidos DEIS Acumulado']=deis['Fallecidos DEIS Diarios'].cumsum()
deis['Fallecidos DEIS (PCR+) x Millón/Hab.']=deis['Fallecidos DEIS (PCR+) Acumulado']/(popTOT/1000000)
deis['Fallecidos DEIS (Covid Sospechoso) x Millón/Hab.']=deis['Fallecidos DEIS (Covid Sospechoso) Acumulado']/(popTOT/1000000)
deis['Fallecidos DEIS x Millón/Hab.']=deis['Fallecidos DEIS Acumulado']/(popTOT/1000000)


DEIS=deis.merge(fr,on='Fecha',how='left')
#----------------------------DEIS--------------------------------------------------------------------

In [10]:
def melt(df,id_var=['Fecha'],Vars=['Fallecidos Nuevos Región Metropolitana (DEIS)',
       'Fallecidos Nuevos Resto de Chile (DEIS)',
       'Fallecidos Acumulados Región Metropolitana (DEIS)',
       'Fallecidos Acumulados Resto de Chile (DEIS)']):
    df=df[Vars+id_var]
    df=df.melt(id_vars=id_var)
    
    return df

In [11]:
#----------------------------ICOVID--------------------------------------------------------------------

#casos nuevos según fecha de inicio de síntomas (icovid)
s="producto53"
n="producto53_confirmados_"
nuevos=pd.read_sql_table(n, con=cnx,schema=s)
nuevos=pd.DataFrame(nuevos.groupby('fecha')['Promedio'].sum())
nuevos.columns=['Casos Nuevos (ICOVID)']
nuevos['Incidencia Nuevos (ICOVID) x 100 K Hab.']=nuevos['Casos Nuevos (ICOVID)']/popTOT*100000
nuevos['Casos Acumulados (ICOVID)']=nuevos['Casos Nuevos (ICOVID)'].cumsum()

#casos activos según fecha de inicio de síntomas (icovid)
s="producto53"
n="producto53_confirmados_regionale_casos_activos"
activos=pd.read_sql_table(n, con=cnx,schema=s)
activos=pd.DataFrame(activos.groupby('fecha')['Promedio'].sum())
activos.columns=['Casos Activos (ICOVID)']
activos['Incidencia Activos (ICOVID) x 100 K Hab.']=activos['Casos Activos (ICOVID)']/popTOT*100000

#activos regional icovid
s="producto53"
n="producto53_confirmados_regionale_casos_activos"
ar=pd.read_sql_table(n, con=cnx,schema=s)
ar=pd.DataFrame(ar.groupby(['fecha','Region_Metropolitana'])[['Poblacion','Promedio']].sum()).reset_index()
ar=ar.pivot(index='fecha',columns=['Region_Metropolitana']).T.reset_index().T
ar.columns=ar.iloc[0].astype(str)+' '+ar.iloc[1].astype(str)+' Casos Activos (ICOVID)'
ar=ar.iloc[3:].reset_index()
ar=ar.rename(columns={'fecha':'Fecha'})

#nuevos regionales icovid
s="producto53"
n="producto53_confirmados_regionale"
nr=pd.read_sql_table(n, con=cnx,schema=s)
nr=pd.DataFrame(nr.groupby(['fecha','Region_Metropolitana'])[['Poblacion','Promedio']].sum()).reset_index()
nr=nr.pivot(index='fecha',columns=['Region_Metropolitana']).T.reset_index().T
nr.columns=nr.iloc[0].astype(str)+' '+nr.iloc[1].astype(str)+' Casos Nuevos (ICOVID)'
nr=nr.iloc[3:].reset_index()
nr=nr.rename(columns={'fecha':'Fecha'})

#icovid consolidado
icovid=pd.concat([nuevos,activos],axis=1)
icovid=icovid.reset_index()
icovid=icovid.rename(columns={'fecha':'Fecha'})

icovid=icovid.merge(ar,how='left',on='Fecha').merge(nr,how='left',on='Fecha')
#----------------------------ICOVID--------------------------------------------------------------------

In [12]:
#----------------------------MINSAL--------------------------------------------------------------------

#casos MINSAL
s="producto5"
n="producto5_TotalesNacionales_T"
minsal=pd.read_sql_table(n, con=cnx,schema=s)
minsal=minsal.drop('index',axis=1)
minsal.columns=[x+' (MINSAL)' for x in minsal.columns]
minsal=minsal.rename(columns={'Fecha (MINSAL)': 'Fecha'})
minsal['Fecha']=pd.to_datetime(minsal['Fecha'],errors='coerce')

#casos MINSAL regionales
s="producto3"
n="producto3_TotalesPorRegion_std"
minsalr=pd.read_sql_table(n, con=cnx,schema=s)
#minsal=minsal.drop('index',axis=1)
#minsal.columns=[x+' (MINSAL)' for x in minsal.columns]
#minsal=minsal.rename(columns={'Fecha (MINSAL)': 'Fecha'})
#minsal['Fecha']=pd.to_datetime(minsal['Fecha'],errors='coerce')

#casos nuevos regional minsal
s="producto13"
n="producto13_CasosNuevosCumulativo_std"
nrm=pd.read_sql_table(n, con=cnx,schema=s)
nrm=nrm[nrm['Region']!='Total']
nrm=pd.DataFrame(nrm.groupby(['Fecha','Region_Metropolitana'])['Total'].sum()).reset_index()
nrm=nrm.pivot(index='Fecha',columns=['Region_Metropolitana']).T.reset_index().T
nrm.columns='Casos Nuevos (MINSAL) '+nrm.iloc[1].astype(str)
nrm=nrm.iloc[3:].reset_index()

#casos activos por fecha inicio de sintomas regional (minsal)
s="producto19"
n="producto19_CasosActivosPorComuna_std"
arms=pd.read_sql_table(n, con=cnx,schema=s)
arms=arms.fillna(0)
arms=pd.DataFrame(arms.groupby(['Fecha','Region_Metropolitana'])['Casos activos'].sum()).reset_index()
arms=arms.pivot(index='Fecha',columns=['Region_Metropolitana']).T.reset_index().T
arms.columns='Casos Activos (MINSAL) '+arms.iloc[1].astype(str)
arms=arms.iloc[3:].reset_index()

#activos regional minsal
s="producto25"
n="producto25_CasosActualesPorComuna_std"
arm=pd.read_sql_table(n, con=cnx,schema=s)
arm=arm[arm['Region']!='Total']
arm=pd.DataFrame(arm.groupby(['Fecha','Region_Metropolitana'])[['Casos actuales']].sum()).reset_index()
arm=arm.pivot(index='Fecha',columns=['Region_Metropolitana']).T.reset_index().T
arm.columns=arm.iloc[0].astype(str)+' '+arm.iloc[1].astype(str)+' (MINSAL)'
arm=arm.iloc[3:].reset_index()
#arm=arm.rename(columns={'fecha':'Fecha'})

minsal=minsal.merge(arm,how='left',on='Fecha').merge(nrm,how='left',on='Fecha').merge(arms,how='left',on='Fecha')
#----------------------------MINSAL--------------------------------------------------------------------

In [14]:
#movilidad
s="movilidad"
n="isci_entel"
arms=pd.read_sql_table(n, con=cnx,schema=s)
mov=pd.read_sql_table(n, schema=s,con=cnx)
mov_chile=pd.DataFrame(mov.groupby('date')['PERSONAS_MOVILIZADAS'].sum()).reset_index().rename(columns={'date':'Fecha'})
mov_rm=pd.DataFrame(mov[mov['REGION']=='13'].groupby('date')['PERSONAS_MOVILIZADAS'].sum()).reset_index().rename(columns={'date':'Fecha',
                                                                                                                          'PERSONAS_MOVILIZADAS':'PERSONAS_MOVILIZADAS_RM'})
mov_resto_chile=pd.DataFrame(mov[mov['REGION']!='13'].groupby('date')['PERSONAS_MOVILIZADAS'].sum()).reset_index().rename(columns={'date':'Fecha',
                                                                                                                                  'PERSONAS_MOVILIZADAS':'PERSONAS_MOVILIZADAS_RESTO_CHILE'})
mov=mov_chile.merge(mov_rm,on='Fecha',how='outer').merge(mov_resto_chile,on='Fecha',how='outer')

In [15]:
mov.head()

,Fecha,PERSONAS_MOVILIZADAS,PERSONAS_MOVILIZADAS_RM,PERSONAS_MOVILIZADAS_RESTO_CHILE
0,2020-03-07,13614325.550,6191274.730,7423050.820
1,2020-03-14,13595685.090,6187748.430,7407936.660
2,2020-03-21,12699112.270,5761242.670,6937869.600
3,2020-03-28,11036079.810,5008535.730,6027544.080
4,2020-04-04,10945147.930,4866717.690,6078430.240


In [16]:
#cuarentenas
s="cuarentenas"
n="cuarentenas"
cuarentenas=pd.read_sql_table(n, con=cnx,schema=s)

In [17]:
cuarentenas

,index,Fecha,CUT,COD_PASO,REG,COMUNA,Fecha2,T_POB
0,0,2020-07-24,1101,1,Tarapacá,Iquique,2020-07-24 00:00:00,191468.000
1,1,2020-07-25,1101,1,Tarapacá,Iquique,2020-07-25 00:00:00,191468.000
2,2,2020-07-26,1101,1,Tarapacá,Iquique,2020-07-26 00:00:00,191468.000
3,3,2020-07-27,1101,1,Tarapacá,Iquique,2020-07-27 00:00:00,191468.000
4,4,2020-07-28,1101,1,Tarapacá,Iquique,2020-07-28 00:00:00,191468.000
...,...,...,...,...,...,...,...,...
64013,64013,2020-04-18,16103,0.0,Ñuble,Chillán Viejo,18-abr,30907.000
64014,64014,2020-04-19,16103,0.0,Ñuble,Chillán Viejo,19-abr,30907.000
64015,64015,2020-04-20,16103,0.0,Ñuble,Chillán Viejo,20-abr,30907.000
64016,64016,2020-04-21,16103,0.0,Ñuble,Chillán Viejo,21-abr,30907.000


In [21]:
pob_cuarentena=cuarentenas.groupby(['Fecha','COD_PASO'])['T_POB'].sum().reset_index()
pob_cuarentena=pob_cuarentena.pivot(columns='COD_PASO',values='T_POB',index='Fecha').reset_index()
pob_cuarentena.columns=[str(x)+'_CUA' if x is not 'Fecha' else x for x in pob_cuarentena.columns]

<>:3: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:3: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-21-5540f1c347e8>:3: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  pob_cuarentena.columns=[str(x)+'_CUA' if x is not 'Fecha' else x for x in pob_cuarentena.columns]


In [18]:
pob_cuarentena_rm=cuarentenas[cuarentenas['REG']=='Metropolitana de Santiago'].groupby(['Fecha','COD_PASO'])['T_POB'].sum().reset_index()
pob_cuarentena_rm=pob_cuarentena_rm.pivot(columns='COD_PASO',values='T_POB',index='Fecha').reset_index()
pob_cuarentena_rm.columns=[str(x)+'_CUA_RM' if x is not 'Fecha' else x for x in pob_cuarentena_rm.columns]

<>:3: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:3: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-18-86aae4bd7312>:3: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  pob_cuarentena_rm.columns=[str(x)+'_CUA_RM' if x is not 'Fecha' else x for x in pob_cuarentena_rm.columns]


In [19]:
pob_cuarentena_rCH=cuarentenas[cuarentenas['REG']!='Metropolitana de Santiago'].groupby(['Fecha','COD_PASO'])['T_POB'].sum().reset_index()
pob_cuarentena_rCH=pob_cuarentena_rCH.pivot(columns='COD_PASO',values='T_POB',index='Fecha').reset_index()
pob_cuarentena_rCH.columns=[str(x)+'_CUA_RCH' if x is not 'Fecha' else x for x in pob_cuarentena_rCH.columns]

<>:3: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:3: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-19-a32158d47903>:3: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  pob_cuarentena_rCH.columns=[str(x)+'_CUA_RCH' if x is not 'Fecha' else x for x in pob_cuarentena_rCH.columns]


In [22]:
pob_cuarentena=pob_cuarentena.merge(pob_cuarentena_rm,on='Fecha',how='outer').merge(pob_cuarentena_rCH,on='Fecha',how='outer')

In [23]:
pob_cuarentena

,Fecha,0.0_CUA,1_CUA,2_CUA,3_CUA,4_CUA,0.0_CUA_RM,1_CUA_RM,2_CUA_RM,3_CUA_RM,4_CUA_RM,0.0_CUA_RCH,1_CUA_RCH,2_CUA_RCH,3_CUA_RCH,4_CUA_RCH
0,2020-01-06,NaN,1205767.000,9969298.000,5450491.000,948309.000,NaN,NaN,5161467.000,1951341.000,NaN,NaN,1205767.000,4807831.000,3499150.000,948309.000
1,2020-01-07,NaN,1205767.000,9969298.000,5450491.000,948309.000,NaN,NaN,5161467.000,1951341.000,NaN,NaN,1205767.000,4807831.000,3499150.000,948309.000
2,2020-01-08,NaN,1205767.000,9969298.000,5450491.000,948309.000,NaN,NaN,5161467.000,1951341.000,NaN,NaN,1205767.000,4807831.000,3499150.000,948309.000
3,2020-03-13,523.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,523.000,NaN,NaN,NaN,NaN
4,2020-03-14,523.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,523.000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,2021-01-13,NaN,1339071.000,9887949.000,6036658.000,310187.000,NaN,NaN,5161467.000,1951341.000,NaN,NaN,1339071.000,4726482.000,4085317.000,310187.000
297,2021-01-14,NaN,1339071.000,9887949.000,6036658.000,310187.000,NaN,NaN,5161467.000,1951341.000,NaN,NaN,1339071.000,4726482.000,4085317.000,310187.000
298,2021-01-15,NaN,1339071.000,9887949.000,6036658.000,310187.000,NaN,NaN,5161467.000,1951341.000,NaN,NaN,1339071.000,4726482.000,4085317.000,310187.000
299,2021-01-16,NaN,1339071.000,9887949.000,6036658.000,310187.000,NaN,NaN,5161467.000,1951341.000,NaN,NaN,1339071.000,4726482.000,4085317.000,310187.000


In [24]:
#s='@perez'
#n='nacional@perez'
#nacional=pd.read_sql_table(n, con=cnx,schema=s)

In [25]:
#nacional.sort_values(by='Fecha').dropna(subset=['Fecha'])

,index,Fecha,Test_PCR_notificados_acumulados,Laboratorios_que_reportan_en_esa_fecha,Confirmados_acumulados_totales__incluye_asintomáticos_,Confirmados_asintomáticos_acumulados_totales,Muertes_acumuladas_totales,Pacientes_actualmente_en_UCI,Pacientes_covid_positivo_actualmenteen_cama_básica,Pacientescovid_positivoactualmenteen_cama_media,...,Pacientes_sospechosos_acumulados_en_Epivigila,Confirmados_acumulados_ingresados_en_Epivigila,Pacientes_acumulados_cn_info_de_ingreso_a_hospital,Pacientes_acumulados_cn_info_de_ingreso_a_UTI/UCI,Confirmados_dossemanas_atrasmenos_muertoshasta_día_actual,Nuevos_confirmados_dos_semanas_atrás_menos_nuevos_muertos_hoy,Camas UCI Desocupadas,VMI Desocupadas,VMI no-COVID,UCI no-COVID
0,0,2020-03-06,None,NaN,7.000,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN
1,1,2020-03-07,None,NaN,9.000,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN
2,2,2020-03-08,None,NaN,12.000,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN
3,3,2020-03-09,None,NaN,17.000,NaN,0.000,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN
4,4,2020-03-10,None,NaN,23.000,NaN,0.000,NaN,NaN,NaN,...,270.000,18.000,NaN,None,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,235,2020-10-27,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN
236,236,2020-10-28,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN
237,237,2020-10-29,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN
238,238,2020-10-30,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN


In [26]:
master=DEIS.merge(icovid,on='Fecha',how='outer').merge(minsal,on='Fecha',
                                                       how='outer').merge(mov,on='Fecha',
                                                                          how='outer').merge(pob_cuarentena,on='Fecha',how='outer')

In [27]:
master=master.sort_values(by='Fecha')

In [28]:
master.columns=[x.replace('x','').replace('.','').replace('(','').replace(')','') for x in master.columns]

In [61]:
#Versión pivoteada
master_melted=melt(master,id_var=['Fecha'],Vars=[x for x in master.columns if x!='Fecha'])
master_melted['Nivel']='Nacional'
master_melted.loc[master_melted['variable'].str.contains('RM'),'Nivel']='RM'
master_melted.loc[master_melted['variable'].str.contains('Metropolitana'),'Nivel']='RM'
master_melted.loc[master_melted['variable'].str.contains('Resto'),'Nivel']='Resto de Chile'
master_melted.loc[master_melted['variable'].str.contains('RCH'),'Nivel']='Resto de Chile'
master_melted.loc[master_melted['variable'].str.contains('RESTO_CHILE'),'Nivel']='Resto de Chile'

In [63]:
name='consolidado_nacional'
schema='nacional'
if not cnx.dialect.has_schema(cnx, schema):
    print('schema '+schema+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(schema))
else:
    print('schema '+schema+' exists, will not be created')
master.to_sql(name, schema=schema,con=cnx,if_exists='replace')
name='consolidado_nacional_melted'
master_melted.to_sql(name, schema=schema,con=cnx,if_exists='replace')

schema nacional exists, will not be created


In [64]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_consolidado_nacional.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/consolidado_nacional.ipynb')

0

In [65]:
master_melted['variable'].unique()

array(['Fallecidos DEIS Diarios', 'Fallecidos DEIS Diarios PCR+',
       'Fallecidos DEIS Diarios Covid Sospechoso',
       'Fallecidos DEIS PCR+ Acumulado',
       'Fallecidos DEIS Covid Sospechoso Acumulado',
       'Fallecidos DEIS Acumulado', 'Fallecidos DEIS PCR+  Millón/Hab',
       'Fallecidos DEIS Covid Sospechoso  Millón/Hab',
       'Fallecidos DEIS  Millón/Hab',
       'Fallecidos Nuevos Región Metropolitana DEIS',
       'Fallecidos Nuevos Resto de Chile DEIS',
       'Fallecidos Acumulados Región Metropolitana DEIS',
       'Fallecidos Acumulados Resto de Chile DEIS', 'Casos Nuevos ICOVID',
       'Incidencia Nuevos ICOVID  100 K Hab', 'Casos Acumulados ICOVID',
       'Casos Activos ICOVID', 'Incidencia Activos ICOVID  100 K Hab',
       'Poblacion Región Metropolitana Casos Activos ICOVID',
       'Poblacion Resto de Chile Casos Activos ICOVID',
       'Promedio Región Metropolitana Casos Activos ICOVID',
       'Promedio Resto de Chile Casos Activos ICOVID',
       'Pob

In [ ]:
       Incidencia Nuevos ICOVID  100 K Hab,
    Casos Acumulados ICOVID,
       Casos Activos ICOVID,
        Incidencia Activos ICOVID  100 K Hab,
       Casos nuevos con sintomas MINSAL, 
    Casos totales MINSAL,
       Casos recuperados MINSAL, 
        Fallecidos MINSAL,
       Casos activos MINSAL, 
    Casos nuevos sin sintomas MINSAL,
       Casos nuevos totales MINSAL, 
        Casos activos por FD MINSAL,
       Casos activos por FIS MINSAL, 
    Casos recuperados por FIS MINSAL,
       Casos recuperados por FD MINSAL,
       Casos confirmados recuperados MINSAL,
       Casos activos confirmados MINSAL,
       Casos probables acumulados MINSAL,
       Casos activos probables MINSAL,
       Casos nuevos sin notificar MINSAL,
       Casos \nactivos confirmados MINSAL,
        Casos Nuevos ICOVID